In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
print("hi")

In [ ]:
from os import path

filePath = 'data-normalized/'
fileCount = 0
scanData = []
scanNP_list =[]

yList = []
vve = pd.read_csv("VVE-clinical-devset.csv")
for index, row in vve.iterrows():
    fileName = row['ScanFileName']
    fnLen = len(fileName)
    modFileName = fileName[0:fnLen-4] + "_norm.csv"
    scanFile = filePath + modFileName
    if path.exists(scanFile):      
        yList.append(row['VVE_NumMeasuresAbnormal'])
        #dataframeY = dataframeY.append({'VVEScore':row['VVE_NumMeasureAbnormal']})
        fileCount += 1
        scanContents = pd.read_csv(scanFile)
        scanNP = scanContents.to_numpy()
        scanData.append(scanContents)  
        scanNP_list.append(scanNP)
        dfx = pd.DataFrame(scanNP)
        dfy = dfx.apply(lambda x: x.str.strip() if isinstance(x, str) else x).replace('', np.nan)
        df = df.append({'patient#': modFileName, 'LeftX': dfy[1], 'LeftY': dfy[2], 'LeftVelX': dfy[3], 'LeftVelY': dfy[4], 'RightX': dfy[5], 
                        'RightY': dfy[6], 'RightVelX': dfy[7], 'RightVelY': dfy[8], 'LeftPupilRadius': dfy[9], 
                        'RightPupilRadius': dfy[10], "LeftRadialVel": dfy[11], 'LeftRadialAcc': dfy[12], 
                        'LeftRadialJerk': dfy[13], 'RightRadialVel': dfy[14], 
                        'RightRadialAcc': dfy[15], 'RightRadialJerk': dfy[16]}, ignore_index=True)
    else:
       print(scanFile)

        #display(dfy)
        #if scanContents['LeftX'].notnull().values.sum() < 1000:
          #print(scanFile + '\n')

display(df)
print(fileCount)

print(index)
print(yList)
#train, test = train_test_split(df, test_size=0.2)



#display(train)

#print(str(fileCount) + ' datafiles found.')
#print(scanData)
#scanArray = np.dstack(scanNP_list)
#np.save(filePath + 'scanArray', scanArray)


In [ ]:
rec_count = len(data)
rec_count

In [ ]:
sequence_length = 10

In [ ]:
def generate_data(X, y, sequence_length = 10, step = 1):
    X_local = []
    y_local = []
    for start in range(0, len(data) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

X_sequence, y = generate_data(data.loc[:, "V1":"V28"].values, data.Class)

In [ ]:
model = keras.Sequential()
model.add(LSTM(100, input_shape = (10, 28)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

In [ ]:
training_size = int(len(X_sequence) * 0.7)

In [ ]:
X_train, y_train = X_sequence[:training_size], y[:training_size]
X_test, y_test = X_sequence[training_size:], y[training_size:]

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=50)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_test_prob = model.predict(X_test, verbose=1)

In [ ]:
y_test_pred = np.where(y_test_prob > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_test_pred)